In [1]:
from utils.segment_hota.map_points_to_segments import map_points_to_segments
from utils.segment_hota.detection_accuracy import detection_accuracy
from utils.segment_hota.association_accuracy import association_accuracy
from utils.segment_hota.types import Polygon, Segment, SegmentDetection
from utils.segment_hota.object_type import object_type

In [2]:
import os
import json
import shapely.wkt
import shapely.wkt
import postgis
import postgis.geometry

In [3]:
with open(os.path.join('/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini/annotation_splitted.json'), 'r') as f:
    ground_truth_split = json.load(f)
    

In [4]:
with open(os.path.join('/data/chanwutk/code/apperception/outputs/segment-tracking-without-de-segment.json'), 'r') as f:
    prediction_split = json.load(f)
# [*prediction_split.values()][0]



In [5]:
[*prediction_split.keys()][0]

'scene-0103-CAM_FRONT'

In [6]:
with open(os.path.join('/data/chanwutk/code/apperception/outputs/segment-tracking-without-de-frame-id.json'), 'r') as f:
    prediction_frame_id_split = json.load(f)
# [*prediction_frame_id_split.values()][0]



In [7]:
[*ground_truth_split.values()][30][1]

[{'token': 'aa493b8204ee4141a610e86a6a0b30dc',
  'instance_token': 'b53f6f12d9b149938350665b4bdfde22',
  'translation': [1847.727, 2533.368, -0.252],
  'size': [2.052, 5.751, 1.998],
  'rotation': [0.9133145965806944, 0.0, 0.0, -0.40725476998144977],
  'category': 'vehicle.truck',
  'location': 'singapore-queenstown',
  'scene_name': 'scene-0796',
  'channels': ['CAM_FRONT'],
  'sample_data_token': '5f413725e3d84fbc987d7f6b521bcf95',
  'timestamp': 1538448745012460},
 {'token': '0e4acb7ad8104c52ba4fb18858e4fd93',
  'instance_token': '7fd4411922dc48bfb610fa548c9313a6',
  'translation': [1807.911, 2559.964, -0.854],
  'size': [2.097, 5.259, 2.124],
  'rotation': [0.4306056128058851, 0.0, 0.0, 0.9025401964566832],
  'category': 'vehicle.car',
  'location': 'singapore-queenstown',
  'scene_name': 'scene-0796',
  'channels': ['CAM_FRONT'],
  'sample_data_token': '5f413725e3d84fbc987d7f6b521bcf95',
  'timestamp': 1538448745012460},
 {'token': 'a1a9886812f54dc184da004455af4bd5',
  'instance_t

In [8]:
[*ground_truth_split.keys()][0]

'scene-0061_CAM_FRONT'

In [9]:
def format_prediction(prediction_frames, predictions):
    out = []
    for fid, pds in zip(prediction_frames, predictions):
        for oid, pd in pds.items():
            road_polygon_info = pd['road_polygon_info']
            if road_polygon_info is None:
                continue
#             print(pd['segment_line'])
#             print(fid)
            out.append(SegmentDetection(
                oid=oid,
                did=str(pd['detection_id']),
                fid=fid,
                timestamp=pd['timestamp'],
                segment=Segment(
                    id=pd['segment_line'],
                    line=shapely.wkb.loads(pd['segment_line'], hex=True),
#                     line='',
                    polygon=Polygon(
                        id=road_polygon_info['id'],
                        polygon=shapely.wkt.loads(road_polygon_info['polygon']),
                        segments=None,
                    ),
                ),
                location='boston-seaport',
                type=pd['type']
            ))
    return out

In [10]:
for name, (sdts, groundtruths) in ground_truth_split.items():
    if len(groundtruths) == 0:
        continue

    locations = set(a['location'] for a in groundtruths)
    assert len(locations) == 1, locations
    if [*locations][0] != 'boston-seaport':
        continue
    
    pd_keys = name.split('_')
    pd_key = f"{pd_keys[0]}-{'_'.join(pd_keys[1:])}"
    
    if pd_key not in prediction_split:
        continue

    predictions = prediction_split[pd_key]
    prediction_frames = prediction_frame_id_split[pd_key]

    formatted_predictions = format_prediction(prediction_frames, predictions)
    formatted_groundtruths = map_points_to_segments(groundtruths)
    
    formatted_groundtruths = [
        gt
        for gt
        in formatted_groundtruths
        if gt.type.startswith('vehicle') and object_type(gt.type) is not None
    ]
    for gt in formatted_groundtruths:
        t = object_type(gt.type)
        assert t is not None, gt.type
        gt.type = t
    
    print(name)
    print(len(formatted_groundtruths), len(groundtruths))
    print('DetA', detection_accuracy  (formatted_predictions, formatted_groundtruths   ) * 100, '%')
    print('AssA', association_accuracy(formatted_predictions, formatted_groundtruths, 0) * 100, '%')
    print()
    print()

result length 431
scene-0103_CAM_FRONT
370 727
154 1608 216
DetA 7.785642062689585 %
AssA 7.079921890754128 %


result length 58
scene-0103_CAM_FRONT_RIGHT
20 264
19 155 1
DetA 10.857142857142858 %
AssA 0 %


result length 128
scene-0103_CAM_FRONT_LEFT
101 269
24 319 77
DetA 5.714285714285714 %
AssA 4.0 %


result length 395
scene-0553_CAM_FRONT
157 623
44 318 113
DetA 9.263157894736842 %
AssA 25.0 %


result length 191
scene-0553_CAM_FRONT_RIGHT
178 251
0 18 178
DetA 0.0 %
AssA 0 %


result length 273
scene-0553_CAM_FRONT_LEFT
137 450
33 410 104
DetA 6.032906764168191 %
AssA 1.6025641025641024 %


result length 281
scene-0655_CAM_FRONT
192 619
43 581 149
DetA 5.5627425614489 %
AssA 2.3452252758400047 %


result length 6
scene-0655_CAM_FRONT_RIGHT
6 393
2 46 4
DetA 3.8461538461538463 %
AssA 0 %


result length 119
scene-0655_CAM_FRONT_LEFT
92 198
14 384 78
DetA 2.941176470588235 %
AssA 8.333333333333332 %


result length 108
scene-0757_CAM_FRONT
102 126
55 432 47
DetA 10.29962546816479